<a href="https://colab.research.google.com/github/rtolps/TensorflowCats2DogsUGATIT/blob/master/WorkingAnimetensorflowproper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UGATIT Tutorial

### Install stuff

Uninstall new tensor flow and install old one

In [ ]:
!pip uninstall -y tensorflow
!pip install opencv-python matplotlib numpy Pillow Flask imutils tensorflow-gpu==1.14.0 awscli

In [ ]:
from platform import python_version

print(python_version())

In [ ]:
!pip install git+https://github.com/onnx/tensorflow-onnx

In [ ]:
!git clone https://github.com/rtolps/TensorflowCats2DogsUGATIT
%cd TensorflowCats2DogsUGATIT

#(Optional) install Bazel

In [ ]:
BAZEL_VERSION = '0.20.0'

In [ ]:
!wget https://github.com/bazelbuild/bazel/releases/download/{BAZEL_VERSION}/bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh

In [ ]:
!chmod +x bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh

In [ ]:
!./bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh

In [ ]:
!bazel

# Import Other Stuff

Import Summary tool


In [ ]:
import tensorflow.tools.graph_transforms as graph_transforms

Import graph freezer

In [ ]:
from tensorflow.python.tools import freeze_graph

# Train

In [ ]:
!python main.py --dataset cat2dog --phase train --light True --iteration 50 --epoch 2

#Test

In [ ]:
!python main.py --dataset cat2dog --phase test --light True

Convert Graph using code from the Github page of anime tensorflow in closed issues

# **Export** (I fixed all of the errors with the script, it outputs frozen graph!!)

In [ ]:
!mkdir graphyfreezy

In [12]:
!python freezeGraph.py --checkpoint_dir checkpoint/UGATIT_light_cat2dog_lsgan_4resblock_6dis_1_1_10_10_1000_sn_smoothing --output output

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

# Clean up frozen graph per [snapchat's requirements](https://lensstudio.snapchat.com/guides/machine-learning/ml-frameworks/export-from-tensorflow/) (this has errors)

In [22]:
!python cleanup.py

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 